# 决策树分类实验（Decision Tree Classification）
本实验将带你从零理解并使用决策树进行分类任务，包括熵、信息增益、模型训练、可视化和参数调节。

## 1. 实验介绍
在本实验中，你将：
- 理解熵（Entropy）与信息增益（Information Gain）的含义
- 使用 sklearn 训练决策树
- 可视化决策边界与树结构
- 比较不同参数对模型的影响

## 2. 实验大纲
- [ 1 - Packages ](#1)
- [ 2 -  Problem Statement](#2)
- [ 3 - Dataset](#3)
  - [ 3.1 One hot encoded dataset](#3.1)
- [ 4 - Decision Tree Refresher](#4)
  - [ 4.1  Calculate entropy](#4.1)
    - [ Exercise 1](#ex01)
  - [ 4.2  Split dataset](#4.2)
    - [ Exercise 2](#ex02)
  - [ 4.3  Calculate information gain](#4.3)
    - [ Exercise 3](#ex03)
  - [ 4.4  Get best split](#4.4)
    - [ Exercise 4](#ex04)
- [ 5 - Building the tree](#5)


<a name="1"></a>
## 1 - 导入相关库（Packages）

首先，让我们运行下面的代码单元来导入本次实验所需的全部库。

- [numpy](https://www.numpy.org) 是 Python 中用于处理矩阵和多维数组的基础科学计算包。
- [matplotlib](https://matplotlib.org) 是一个用于绘制图形的著名 Python 可视化库。
- `utils.py` 文件中包含了本次作业会使用到的一些辅助函数。你**不需要修改**其中的任何代码。


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *

%matplotlib inline

<a name="2"></a>
## 2 - 问题描述（Problem Statement）

假设你正在创办一家种植并销售野生蘑菇的公司。  
- 由于并不是所有蘑菇都可食用，你希望能够根据蘑菇的外部物理属性来判断某个蘑菇是否可食用或有毒。
- 你已经收集了一些现有的数据，可以用于完成这一任务。

你能否利用这些数据来帮助你识别哪些蘑菇可以安全出售？

注意：本实验使用的数据集仅用于 **教学和示例目的**，不应被视为现实中识别可食用蘑菇的指导依据。

---

<a name="3"></a>
## 3 - 数据集（Dataset）

你将从加载本任务所需的数据集开始。你已经收集到的数据如下：

| 帽子颜色（Cap Color） | 茎部形状（Stalk Shape） | 是否独生（Solitary） | 可食用（Edible） |
|:---------------------:|:-----------------------:|:--------------------:|:----------------:|
|        Brown         |        Tapering         |         Yes          |        1         |
|        Brown         |        Enlarging        |         Yes          |        1         |
|        Brown         |        Enlarging        |         No           |        0         |
|        Brown         |        Enlarging        |         No           |        0         |
|        Brown         |        Tapering         |         Yes          |        1         |
|         Red          |        Tapering         |         Yes          |        0         |
|         Red          |        Enlarging        |         No           |        0         |
|        Brown         |        Enlarging        |         Yes          |        1         |
|         Red          |        Tapering         |         No           |        1         |
|        Brown         |        Enlarging        |         No           |        0         |

你一共有 **10 个蘑菇样本（examples）**，每个样本包含：

- 三个特征（features）  
  - 帽子颜色 Cap Color（`Brown` 或 `Red`）  
  - 茎部形状 Stalk Shape（`Tapering` 或 `Enlarging`）  
  - 是否独生 Solitary（`Yes` 或 `No`）  
- 一个标签（label）  
  - Edible（可食用，`1` 表示可食用，`0` 表示有毒）

---

<a name="3.1"></a>
### 3.1 One-hot 编码后的数据集（One hot encoded dataset）

为了便于后续算法实现，我们对特征进行了 **One-hot 编码**（即将分类特征转成 0 或 1 的二值特征）。

| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

因此：

- `X_train` 中每个样本包含三个特征：  
  - **Brown Color**：`1` 表示帽子颜色为 Brown，`0` 表示 Red  
  - **Tapering Shape**：`1` 表示茎部形状为 Tapering，`0` 表示 Enlarging  
  - **Solitary**：`1` 表示 Yes，`0` 表示 No  

- `y_train` 表示蘑菇是否可食用  
  - `y = 1` → 可食用  
  - `y = 0` → 有毒  



In [ ]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

#### 查看变量（View the variables）

让我们更熟悉一下你的数据集。  
- 一个很好的开始方式就是直接打印出每个变量，并查看它们包含的内容。

下面的代码将打印 `X_train` 的前几个元素，以及该变量的类型。


In [ ]:
print("First few elements of X_train:\n", X_train[:5])
print("Type of X_train:",type(X_train))

对y_train也做同样的操作。

In [ ]:
print("First few elements of y_train:", y_train[:5])
print("Type of y_train:",type(y_train))

#### 检查变量的维度（Check the dimensions of your variables）

另一种熟悉数据集的有用方式是查看它们的维度（dimensions）。

请打印 `X_train` 和 `y_train` 的形状（shape），并观察你的数据集中有多少个训练样本。


In [ ]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

<a name="4"></a>
## 4 - 决策树复习（Decision Tree Refresher）

在本实验练习中，你将基于提供的数据集构建一棵决策树。

- 回顾一下，构建决策树的步骤如下：
    - 在根节点（root node）开始，使用全部样本
    - 计算在所有可能特征上进行划分（split）的信息增益（information gain），并选择信息增益最大的那个特征
    - 根据所选特征对数据集进行划分，并在树中创建左、右分支
    - 持续重复划分过程，直到满足停止条件（stopping criteria）
  
- 在本次实验中，你将实现以下函数，这些函数将允许你基于信息增益最高的特征，将某个节点划分成左、右子分支：
    - 计算某个节点的熵（entropy）
    - 使用给定特征将节点处的数据集划分为左、右子集
    - 计算基于某特征进行划分所获得的信息增益
    - 选择使信息增益最大的特征
    
- 然后，我们将使用你实现的这些辅助函数，通过不断重复划分过程，构建一棵决策树，直到达到停止条件为止。
    - 在本实验中，我们选择的停止条件是：**将最大深度（max depth）设为 2**


<a name="4.1"></a>
### 4.1  计算熵（Calculate entropy）

首先，你将编写一个名为 `compute_entropy` 的辅助函数，用来计算某个节点的熵（entropy，也是不纯度 impurity 的度量）。

- 该函数接收一个 numpy 数组（`y`），其中每个元素表示该节点中的样本是否可食用：  
  - `1` 表示可食用（edible）  
  - `0` 表示有毒（poisonous）

请完成下面的 `compute_entropy()` 函数，使其能够：

* 计算 \(p_1\)，即该节点中可食用样本的比例  
  （也就是 `y` 中值为 `1` 的样本占比）

* 熵的计算公式如下：

$$
H(p_1) = -p_1 \log_2(p_1) - (1 - p_1)\log_2(1 - p_1)
$$

* 注意事项  
    * 对数的底数为 \(2\)  
    * 在实际实现中，规定 $0 / log_2(0) = 0$。也就是说，如果 `p_1 = 0` 或 `p_1 = 1`，则熵应设为 `0`
    * 请确保节点中的数据非空（即 `len(y) != 0`）。如果为空，则返回 `0`

<a name="ex01"></a>
### 练习 1（Exercise 1）

请根据前面的说明完成 `compute_entropy()` 函数的实现。

如果你在实现过程中遇到困难，可以查看本单元后给出的提示。


In [ ]:
# UNQ_C1
# GRADED FUNCTION: compute_entropy

def compute_entropy(y):
    """
    Computes the entropy for 
    
    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)
       
    Returns:
        entropy (float): Entropy at that node
        
    """
    # You need to return the following variables correctly
    entropy = 0.
    
    ### START CODE HERE ###
           
    ### END CODE HERE ###        
    
    return entropy

<details>
  <summary><font size="3" color="darkgreen"><b>点击查看提示（Click for hints）</b></font></summary>
    
    
   * 计算 `p1` 的提示
       * 你可以通过 `y[y == 1]` 获取 `y` 中值为 `1` 的样本子集
       * 可以用 `len(y)` 获取 `y` 中样本总数
   * 计算 `entropy` 的提示
       * <a href="https://numpy.org/doc/stable/reference/generated/numpy.log2.html">np.log2</a> 可以用来对 numpy 数组计算以 2 为底的对数
       * 如果 `p1` 等于 0 或 1，请记得将熵设为 `0`（处理 0 log 0 的情况）


</details>
    


<details>
        <summary><font size="2" color="darkblue"><b> 点击查看更多提示（Click for more hints）</b></font></summary>
        
    * 下面是你可以参考的函数实现结构示例：
    
```python
    def compute_entropy(y):
        
        # 你需要正确返回以下变量
        entropy = 0.

        ### START CODE HERE ###
        if len(y) != 0:
            # 你的代码：计算可食用样本（值为 1）的比例 p1
            p1 =

            # 当 p1 = 0 或 1 时，熵应该为 0（处理 0log0）
            if p1 != 0 and p1 != 1:
                # 你的代码：根据上面的公式计算熵
                entropy = 
            else:
                entropy = 0.
        ### END CODE HERE ###        

        return entropy
```

你可以运行下面的测试代码来检查你的实现是否正确：

In [ ]:
# Compute entropy at the root node (i.e. with all examples)
# Since we have 5 edible and 5 non-edible mushrooms, the entropy should be 1"

print("Entropy at root node: ", compute_entropy(y_train)) 

# UNIT TESTS
compute_entropy_test(compute_entropy)

<a name="4.2"></a>
### 4.2  划分数据集（Split dataset）

接下来，你将编写一个名为 `split_dataset` 的辅助函数。  
该函数接收某个节点处的数据以及要用于划分的特征，并将数据划分为左分支与右分支。  
在实验的后续部分，你将实现代码来计算这个划分的质量（即信息增益）。

- 该函数接收训练数据、该节点中样本的索引列表（`node_indices`），以及用于划分的特征编号。
- 它根据该特征对数据进行划分，并返回左分支和右分支对应的索引子集。
- 例如，假设我们当前位于根节点（因此 `node_indices = [0,1,2,3,4,5,6,7,8,9]`），并选择使用特征 `0`（表示样本是否具有棕色菌盖）进行划分：
    - 则函数输出为：  
      `left_indices = [0,1,2,3,4,7,9]`  
      `right_indices = [5,6,8]`

| 索引（Index） | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:-------------:|:---------:|:--------------------:|:--------:|:------:|
|       0       |     1     |           1          |     1    |    1   |
|       1       |     1     |           0          |     1    |    1   |
|       2       |     1     |           0          |     0    |    0   |
|       3       |     1     |           0          |     0    |    0   |
|       4       |     1     |           1          |     1    |    1   |
|       5       |     0     |           1          |     1    |    0   |
|       6       |     0     |           0          |     0    |    0   |
|       7       |     1     |           0          |     1    |    1   |
|       8       |     0     |           1          |     0    |    1   |
|       9       |     1     |           0          |     0    |    0   |

<a name="ex02"></a>
### 练习 2（Exercise 2）

请完成下面提供的 `split_dataset()` 函数。

- 对于 `node_indices` 中的每一个索引：
    - 如果该样本在所选特征上的值为 `1`，则将该索引加入 `left_indices`
    - 如果值为 `0`，则将该索引加入 `right_indices`

如果你在实现过程中卡住，可以查看该代码单元之后提供的提示。


In [ ]:
# UNQ_C2
# GRADED FUNCTION: split_dataset

def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches
    
    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (ndarray):  List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on
    
    Returns:
        left_indices (ndarray): Indices with feature value == 1
        right_indices (ndarray): Indices with feature value == 0
    """
    
    # You need to return the following variables correctly
    left_indices = []
    right_indices = []
    
    ### START CODE HERE ###
           
    ### END CODE HERE ###
        
    return left_indices, right_indices

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Here's how you can structure the overall implementation for this function

```python 
    def split_dataset(X, node_indices, feature):
    
        ### You need to return the following variables correctly
        left_indices = []
        right_indices = []

        #### START CODE HERE ###
     
        ### Go through the indices of examples at that node
        for i in node_indices:   
            if # Your code here to check if the value of X at that index for the feature is 1
                left_indices.append(i)
            else:
                right_indices.append(i)
        #### END CODE HERE ###
        
        return left_indices, right_indices
```


</details>

<details>
    <summary><font size="2" color="darkblue"><b> Click for more hints</b></font></summary>
    The condition is <code>if X[i][feature] == 1:</code>.
        
</details>


现在，让我们使用下面的代码块来检查你的实现是否正确。  
我们将尝试在根节点（其中包含所有样本）使用特征 0（Brown Cap）对数据集进行划分，就像我们之前讨论的那样。

In [ ]:
root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Feel free to play around with these variables
# The dataset only has three features, so this value can be 0 (Brown Cap), 1 (Tapering Stalk Shape) or 2 (Solitary)
feature = 0

left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("Left indices: ", left_indices)
print("Right indices: ", right_indices)

# UNIT TESTS    
split_dataset_test(split_dataset)

<a name="4.3"></a>
### 4.3  计算信息增益（Calculate information gain）

接下来，你将编写一个名为 `information_gain` 的函数。  
该函数接收训练数据、某节点中的样本索引，以及用于划分的特征编号，并返回基于该划分所获得的信息增益。

<a name="ex03"></a>
### 练习 3（Exercise 3）

请完成下面的 `compute_information_gain()` 函数，以计算：

$$
\text{Information Gain} = H(p_1^{\text{node}}) - \big( 
w^{\text{left}} H(p_1^{\text{left}}) \;+\; 
w^{\text{right}} H(p_1^{\text{right}})
\big)
$$

其中：

- $H(p_1^{\text{node}})$：当前节点的数据熵  
- $H(p_1^{\text{left}})$ 与 $H(p_1^{\text{right}})$：基于划分后得到的左、右子节点的熵  
- $w^{\text{left}}$ 与 $w^{\text{right}}$：左、右子节点中样本数占该节点样本总数的比例  

注意：

- 你可以直接使用之前实现的 `compute_entropy()` 函数来计算熵  
- 我们已经提供了部分起始代码，其中使用了你在上一部分实现的 `split_dataset()` 函数来对数据集进行划分

如果你在实现过程中遇到困难，可以查看该代码单元后给出的提示。


In [ ]:
# UNQ_C3
# GRADED FUNCTION: compute_information_gain

def compute_information_gain(X, y, node_indices, feature):
    
    """
    Compute the information of splitting the node on a given feature
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
   
    Returns:
        cost (float):        Cost computed
    
    """    
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    # Some useful variables
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    # You need to return the following variables correctly
    information_gain = 0
    
    ### START CODE HERE ###
    
    # Weights 
    
    #Weighted entropy
     
    #Information gain                                                   
    
    ### END CODE HERE ###  
    
    return information_gain

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Here's how you can structure the overall implementation for this function
```python 
    def compute_information_gain(X, y, node_indices, feature):
        # Split dataset
        left_indices, right_indices = split_dataset(X, node_indices, feature)

        # Some useful variables
        X_node, y_node = X[node_indices], y[node_indices]
        X_left, y_left = X[left_indices], y[left_indices]
        X_right, y_right = X[right_indices], y[right_indices]

        # You need to return the following variables correctly
        information_gain = 0

        ### START CODE HERE ###
        # Your code here to compute the entropy at the node using compute_entropy()
        node_entropy = 
        # Your code here to compute the entropy at the left branch
        left_entropy = 
        # Your code here to compute the entropy at the right branch
        right_entropy = 

        # Your code here to compute the proportion of examples at the left branch
        w_left = 
        
        # Your code here to compute the proportion of examples at the right branch
        w_right = 

        # Your code here to compute weighted entropy from the split using 
        # w_left, w_right, left_entropy and right_entropy
        weighted_entropy = 

        # Your code here to compute the information gain as the entropy at the node
        # minus the weighted entropy
        information_gain = 
        ### END CODE HERE ###  

        return information_gain
```

现在，你可以使用下面的代码单元来检查你的实现，并计算在每个特征上进行划分时所得到的信息增益。

In [ ]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)
    
info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)

# UNIT TESTS
compute_information_gain_test(compute_information_gain)

在根节点对特征 “Solitary”（特征编号 = 2）进行划分，会获得最大的的信息增益。因此，它是根节点上最优的划分特征。

<a name="4.4"></a>
### 4.4  获取最佳划分特征（Get best split）

现在，让我们编写一个函数，通过计算每个特征的信息增益（就像我们之前所做的那样），并返回能够产生最大信息增益的那个特征，即最佳划分特征。

<a name="ex04"></a>
### 练习 4（Exercise 4）

请完成下面给出的 `get_best_split()` 函数。

- 该函数接收训练数据以及该节点中所有数据点的索引。
- 函数的输出是：能够产生最大信息增益的特征编号  
    - 你可以使用 `compute_information_gain()` 函数，对每个特征进行遍历，并计算该特征对应的信息增益

如果在实现过程中遇到困难，你可以查看该代码单元之后提供的提示。


In [ ]:
# UNQ_C4
# GRADED FUNCTION: get_best_split

def get_best_split(X, y, node_indices):   
    """
    Returns the optimal feature and threshold value
    to split the node data 
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """    
    
    # Some useful variables
    num_features = X.shape[1]
    
    # You need to return the following variables correctly
    best_feature = -1
    
    ### START CODE HERE ###
       
    ### END CODE HERE ##    
   
    return best_feature

<details>
  <summary><font size="3" color="darkgreen"><b>Click for hints</b></font></summary>
    
    
   * Here's how you can structure the overall implementation for this function
    
```python 
    def get_best_split(X, y, node_indices):   

        # Some useful variables
        num_features = X.shape[1]

        # You need to return the following variables correctly
        best_feature = -1

        ### START CODE HERE ###
        max_info_gain = 0

        # Iterate through all features
        for feature in range(num_features): 
            
            # Your code here to compute the information gain from splitting on this feature
            info_gain = 
            
            # If the information gain is larger than the max seen so far
            if info_gain > max_info_gain:  
                # Your code here to set the max_info_gain and best_feature
                max_info_gain = 
                best_feature = 
        ### END CODE HERE ##    
   
    return best_feature
```
If you're still stuck, check out the hints below.
    
<details>
          <summary><font size="2" color="darkblue"><b> Hint to calculate info_gain</b></font></summary>
        
<code>info_gain = compute_information_gain(X, y, node_indices, feature)</code>
</details>
    
<details>
          <summary><font size="2" color="darkblue"><b>Hint to update the max_info_gain and best_feature</b></font></summary>
           <code>max_info_gain = info_gain</code><br>
           <code>best_feature = feature</code>
</details>
</details>


现在，让我们使用下面的代码单元来检查你所编写函数的实现是否正确。

In [ ]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)

# UNIT TESTS
get_best_split_test(get_best_split)

<a name="5"></a>
## 5 - 构建决策树（Building the tree）

在本节中，我们将使用你在前面实现的那些函数，通过不断选择最佳特征进行划分，来逐步生成一棵决策树，直到达到停止条件（本实验中，最大深度设为 2）。

在这一部分中，你不需要实现任何新代码。


In [ ]:
# Not graded
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.
    
    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree. 
        current_depth (int):    Current depth. Parameter used during recursive call.
   
    """ 

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    tree.append((current_depth, branch_name, best_feature, node_indices))
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)


In [ ]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)